<a href="https://colab.research.google.com/github/nilaynishant/AIMLTutorial/blob/main/04_train_test_data_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run this only if you have not installed the requried packages
!pip install -q rasterio
!pip install -q geopandas
!pip install -q gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [2]:
import os
import geopandas as gpd
import pandas as pd
import rasterio
import git

repo_url = 'https://github.com/nilaynishant/AIMLTutorial.git'  # set the URL of the repository
repo_dir = 'https://github.com/nilaynishant/AIMLTutorial/tree/main/Data'  # set the path to the local repository directory
if not os.path.exists('https://github.com/nilaynishant/AIMLTutorial/tree/main/Data'):
  git.Repo.clone_from(repo_url, repo_dir)
# variables
# Note: labels should be always last column with name "labels"
# Note: Make sure input labels shapefile and input raster have same CRS, otherwise code will not run

# input files
raster_loc = '/content/https:/github.com/nilaynishant/AIMLTutorial/tree/main/Data/Data/Sentinel2_Agartala.tif'
points_loc = '/content/https:/github.com/nilaynishant/AIMLTutorial/tree/main/Data/Data/Training.gpkg'
temp_point_loc = '/content/https:/github.com/nilaynishant/AIMLTutorial/tree/main/Data/Data/temp_y_points.shp'



def get_features(raster_path, shapefile_path):
    # Load the raster data
    with rasterio.open(raster_path) as src:
        # Load the point data shapefile
        points = gpd.read_file(shapefile_path)

        # List to collect data
        data = []

        # Iterate through each point in the GeoDataFrame
        for _, point in points.iterrows():
            # Sample the raster at the point coordinates
            for val in src.sample([(point.geometry.x, point.geometry.y)]):
                # Prepare the record containing the point properties and raster values
                record = point.to_dict()
                # Update record with raster values; assuming raster has multiple bands
                record.update({f'Band_{i+1}_value': val[i] for i in range(len(val))})
                data.append(record)

        # Convert the list of dictionaries to a DataFrame
        return pd.DataFrame(data)

# Specify the path to your raster and shapefile
raster_path = raster_loc
shapefile_path = points_loc

# Extract the features and display as a DataFrame
df_features = get_features(raster_path, shapefile_path)
df_features

x = df_features.iloc[:,2:]
X = x.values
y = df_features.iloc[:,0]
Y = y.values

# Train Test Split

In [3]:
import sklearn
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, stratify = Y)

print(f'X_train Shape: {X_train.shape}\nX_test Shape: {X_test.shape}\ny_train Shape: {y_train.shape}\ny_test Shape:{y_test.shape}')

X_train Shape: (35, 4)
X_test Shape: (16, 4)
y_train Shape: (35,)
y_test Shape:(16,)
